Define environment variables and handler

In [1]:
%run ../env_variables.py

Import required libraries

In [2]:
import logging

from utils import handler as h
import utils.queries as q
import utils.functions as f

In [3]:
# Set up logging

# create logger with 'main'
logger = logging.getLogger('main')
logger.setLevel(logging.DEBUG)
# create file handler which logs even debug messages
fh = logging.FileHandler('main.log')
fh.setLevel(logging.DEBUG)
# create console handler with a higher log level
ch = logging.StreamHandler()
ch.setLevel(logging.ERROR)
# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fh.setFormatter(formatter)
ch.setFormatter(formatter)
# add the handlers to the logger
logger.addHandler(fh)
logger.addHandler(ch)

In [4]:
import pandas as pd

pd.options.display.max_colwidth = 1_000
pd.options.display.max_columns = 1_000

In [5]:
author = 'mergoni'
max_date = '2021-02-01'
file_name_prefix = author + '_scopus_'

## Retrieve original results from Scopus API

In [6]:
original_df = f.retrieve_results_from_list_of_queries(
    list_of_queries = [q.mergoni_scopus_original_query],
    max_date = max_date)

In [7]:
# Drop the column of localization in title, abstract or keywords
# because this data has not yet been retrieved
original_df.drop(
    columns=['localization_in_title_abstract_or_key'],
    inplace=True)

## Retrieve results after applying language-bias-helper

In [8]:
lang_bias_helper_df = f.retrieve_results_from_list_of_queries(
    list_of_queries = [q.mergoni_scopus_lang_bias_helper_query],
    max_date = max_date)

In [9]:
# Drop the column of localization in title, abstract or keywords
# because this data has not yet been retrieved
lang_bias_helper_df.drop(
    columns=['localization_in_title_abstract_or_key'],
    inplace=True)

In [10]:
print(f'Lenght of original df: {len(original_df)}')
print(f'Lenght of language-bias-helper df: {len(lang_bias_helper_df)}')

Lenght of original df: 165
Lenght of language-bias-helper df: 175


## Retrieve results after applying localization-bias-helper

In [11]:
local_bias_helper_df__weird = f.retrieve_results_from_list_of_queries(
    list_of_queries = q.mergoni_scopus_local_bias_helper_queries_weird,
    max_date = max_date)

In [13]:
local_bias_helper_df__non_weird = f.retrieve_results_from_list_of_queries(
    list_of_queries = q.mergoni_scopus_local_bias_helper_queries_non_weird,
    max_date = max_date)

In [15]:
local_bias_helper_df_intersection = local_bias_helper_df__weird[
    local_bias_helper_df__weird['dc:identifier'].isin(
        local_bias_helper_df__non_weird['dc:identifier'])]

In [17]:
local_bias_helper_df_intersection['dc:title']

1                 Development of a global index measuring national policy commitments to HIV prevention and treatment among people who inject drugs
3                                 Assessing regional and global environmental footprints and value added of the largest food producers in the world
4                                  Investigating the energy efficiencies of European countries with super efficiency model and super SBM approaches
14    A conditional nonparametric analysis for measuring the efficiency of regional public healthcare delivery: An application to Greek prefectures
16                               Assessment of cross-border spillover effects of national transport infrastructure plans: An accessibility approach
Name: dc:title, dtype: object

In [18]:
# Remove false positive from WEIRD table
local_bias_helper_df__weird_curated = local_bias_helper_df__weird.drop(
    index=14).reset_index(drop=True)

In [20]:
local_bias_helper_df = pd.concat(
    [local_bias_helper_df__weird_curated,
     local_bias_helper_df__non_weird]
     ).drop_duplicates()

In [22]:
original_df_with_localization = pd.merge(
    original_df,
    local_bias_helper_df,
    how='left',
    on=original_df.columns.to_list()
    ).fillna(False)

In [24]:
original_df_with_localization.agg(
    {'localization_in_title_abstract_or_key': ['sum', 'count'],
     'localization_in_title': ['sum', 'count'],
     }
)

,localization_in_title_abstract_or_key,localization_in_title
sum,96,48
count,165,165


In [25]:
local_bias_helper_df__weird_curated.agg(
    {'localization_in_title_abstract_or_key': ['sum', 'count'],
     'localization_in_title': ['sum', 'count'],
     }
)

,localization_in_title_abstract_or_key,localization_in_title
sum,25,8
count,25,25


In [26]:
local_bias_helper_df__non_weird.agg(
    {'localization_in_title_abstract_or_key': ['sum', 'count'],
     'localization_in_title': ['sum', 'count'],
     }
)

,localization_in_title_abstract_or_key,localization_in_title
sum,75,40
count,75,75


## Results after applying availability-bias-helper

In [ ]:
open_access_df = original_df_with_localization[original_df_with_localization['openaccess'] == 1]
closed_access_df = original_df_with_localization[original_df_with_localization['openaccess'] == 0]

In [ ]:
print(f'Lenght of open access df: {len(open_access_df)}')
print(f'Lenght of closed access df: {len(closed_access_df)}')

## Results after applying publication-bias-helper

In [27]:
pub_bias_helper_df = f.retrieve_results_from_list_of_queries(
    list_of_queries = [q.mergoni_scopus_pub_bias_helper_query],
    max_date = max_date)

In [29]:
pub_bias_helper_df['prism:aggregationType'].value_counts()

prism:aggregationType
Journal                  165
Book                      15
Book Series               13
Conference Proceeding      5
Name: count, dtype: int64

In [ ]:
# lang_bias_helper_df

In [ ]:
# local_bias_helper__non_weird_new_records_df = local_bias_helper_df__non_weird[
#     ~local_bias_helper_df__non_weird['dc:identifier'].isin(original_df['dc:identifier'])
#     ].copy().reset_index(drop=True)

In [ ]:
# local_bias_helper__weird_new_records_df = local_bias_helper_df__weird[
#     ~local_bias_helper_df__weird['dc:identifier'].isin(original_df['dc:identifier'])
#     ].copy().reset_index(drop=True)

In [ ]:
# df_dict = {
#     'original': original_df,
#     'lang_bias_helper': lang_bias_helper_df,
#     'local_bias_helper__non_weird': local_bias_helper_df__non_weird,
#     'local_bias_helper__weird': local_bias_helper_df__weird,
#     'lang_bias_helper_new_records': lang_bias_helper_new_records_df,
#     'local_bias_helper__non_weird_new_records': local_bias_helper__non_weird_new_records_df,
#     'local_bias_helper__weird_new_records': local_bias_helper__weird_new_records_df
#     }

In [ ]:
# if h.save_to_csv:
#     f.export_to_csv(original_df, file_name_prefix + 'original')
#     f.export_to_csv(lang_bias_helper_df, file_name_prefix + 'lang_bias_helper')
#     f.export_to_csv(local_bias_helper_df__weird, file_name_prefix + 'local_bias_helper__weird')
#     f.export_to_csv(local_bias_helper_df__non_weird, file_name_prefix + 'local_bias_helper__non_weird')
#     f.export_to_csv(lang_bias_helper_new_records_df, file_name_prefix + 'lang_bias_helper_new_records')
#     f.export_to_csv(local_bias_helper__non_weird_new_records_df,
#                     file_name_prefix + 'local_bias_helper__non_weird_new_records')
#     f.export_to_csv(local_bias_helper__weird_new_records_df,
#                     file_name_prefix + 'local_bias_helper__weird_new_records')